In [16]:
import pandas as pd
import numpy as np
import os
import random

from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

import xgboost as xgb
from xgboost import plot_importance

import matplotlib
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

In [9]:
CFG = {
    'NBITS': 2048,
    'SEED': 42,
}

In [10]:
import os

print(os.getcwd())

/home/wjstjrals417/Dacon/New_drug_development/Seokmin/src/gbr


In [11]:
from os.path import join
train_path = join('..', '..', '..', 'data', 'total_data', 'train.csv')
test_path = join('..', '..', '..', 'data', 'total_data', 'test.csv')
submission_path = join('..', '..', '..', 'data', 'total_data', 'sample_submission.csv')

In [12]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG['SEED'])

def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=CFG['NBITS'])
        return np.array(fp)
    else:
        return np.zeros((CFG['NBITS'],))

In [13]:
# Load and preprocess data
chembl_data = pd.read_csv(train_path)
train = chembl_data[['Smiles', 'pIC50']]
train['Fingerprint'] = train['Smiles'].apply(smiles_to_fingerprint)

train_x = np.stack(train['Fingerprint'].values)
train_y = train['pIC50'].values

# Split data
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.3, random_state=CFG['SEED'])

/tmp/ipykernel_505329/1984390722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Fingerprint'] = train['Smiles'].apply(smiles_to_fingerprint)


In [17]:
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [18]:
xgb_model.fit(train_x, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.08, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [7]:
# Train GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=CFG['SEED'])
model.fit(train_x, train_y)

GradientBoostingRegressor(random_state=42)

In [20]:
def pIC50_to_IC50(pic50_values):
    return 10 ** (9 - pic50_values)

# Evaluate model
val_y_pred = xgb_model.predict(val_x)
mse = mean_squared_error(pIC50_to_IC50(val_y), pIC50_to_IC50(val_y_pred))
rmse = np.sqrt(mse)

print(f'RMSE: {rmse}')

RMSE: 2200.265036702528


In [25]:
# Predict test data
test = pd.read_csv(test_path)
test['Fingerprint'] = test['Smiles'].apply(smiles_to_fingerprint)
test_x = np.stack(test['Fingerprint'].values)
test_y_pred = xgb_model.predict(test_x)

# Save results
submit = pd.read_csv(submission_path)
submit['IC50_nM'] = pIC50_to_IC50(test_y_pred)
submit.to_csv('./xg_boosting_submit.csv', index=False)

In [21]:
import numpy as np

def calculate_score(y_true, y_pred):
    # IC50(nM) to pIC50 변환
    def to_pIC50(IC50):
        return -np.log10(IC50 * 1e-9)
    
    y_true_pIC50 = to_pIC50(y_true)
    y_pred_pIC50 = to_pIC50(y_pred)
    
    # Normalized RMSE 계산
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    normalized_rmse = rmse / (np.max(y_true) - np.min(y_true))
    
    # Correct Ratio 계산
    absolute_errors_pIC50 = np.abs(y_true_pIC50 - y_pred_pIC50)
    correct_ratio = np.mean(absolute_errors_pIC50 <= 0.5)
    
    # 최종 점수 계산
    A = normalized_rmse
    B = correct_ratio
    score = 0.5 * (1 - min(A, 1)) + 0.5 * B
    
    return score, normalized_rmse, correct_ratio

# 사용 예시
y_true = np.array([100, 200, 300, 400, 500])  # 실제 IC50(nM) 값
y_pred = np.array([110, 190, 280, 420, 510])  # 예측된 IC50(nM) 값

final_score, normalized_rmse, correct_ratio = calculate_score(y_true, y_pred)

print(f"Normalized RMSE: {normalized_rmse:.4f}")
print(f"Correct Ratio: {correct_ratio:.4f}")
print(f"Final Score: {final_score:.4f}")

Normalized RMSE: 0.0371
Correct Ratio: 1.0000
Final Score: 0.9815


In [22]:
final_score, normalized_rmse, correct_ratio = calculate_score(val_y, val_y_pred)

In [23]:
print(f"Normalized RMSE: {normalized_rmse:.4f}")
print(f"Correct Ratio: {correct_ratio:.4f}")
print(f"Final Score: {final_score:.4f}")

Normalized RMSE: 0.1065
Correct Ratio: 1.0000
Final Score: 0.9467
